In [1]:
import numpy as np
from enum import Enum, IntEnum, auto
import typing

from multiprocessing import Pool
import random
import math
import numba
from copy import deepcopy

from chainreaction.game import *
from chainreaction.positions import *
from chainreaction.board import *
from chainreaction.minimax import *
from chainreaction.four_players import *

from tqdm.notebook import tqdm

In [2]:
def get_fresh_board():
    w, h = 3, 3
    state = Board(w, h)
    state.place_atom(0, 0, FourPlayers.P1.value)
    state.place_atom(0, 2, FourPlayers.P2.value)
    state.place_atom(2, 0, FourPlayers.P34.value)
    state.place_atom(2, 2, FourPlayers.P34.value)
    return state


In [5]:
def simulate_game(n=0):
    state = get_fresh_board()
    history = []


    player = FourPlayers.P1
    temp_player = player
    moves_count = 0
    while True:
        before = state.get_copy()
        curr_player = deepcopy(player)
        _, move = minimax_step(state.get_copy(), deepcopy(player), 0)
        moves_count += 1
        print(f'{player} placing on {move}')
        state, player, utilities, terminal = game_step(state, player, move)
        print("New State:")
        print(format_board(state))
        after = state.get_copy()
        history.append((before, after, curr_player))

        if terminal and moves_count>4:
            max_value = max(utilities)
            won_players = [i+1 for i, j in enumerate(utilities) if j == max_value]
            # if len(won_players)==1:
            #     print("Game Won by Player " + str(won_players[0]))
            # else:
            #     won_players_ints = [str(i) for i in won_players]
            #     print("Game Won by Players " + (", ".join(won_players_ints)))
            break

        temp_player = player
        print('=='*20)
    
    return history

In [9]:
a = simulate_game()

FourPlayers.P1 placing on (0, 0)
New State:
[['----' '1-P1' '1-P2']
 ['1-P1' '----' '----']
 ['1-P34' '----' '1-P34']]
FourPlayers.P2 placing on (0, 0)
New State:
[['1-P2' '1-P1' '1-P2']
 ['1-P1' '----' '----']
 ['1-P34' '----' '1-P34']]
FourPlayers.P3 placing on (1, 1)
New State:
[['1-P2' '1-P1' '1-P2']
 ['1-P1' '1-P3' '----']
 ['1-P34' '----' '1-P34']]
FourPlayers.P4 placing on (2, 2)
New State:
[['1-P2' '1-P1' '1-P2']
 ['1-P1' '1-P3' '1-P34']
 ['1-P34' '1-P34' '----']]
FourPlayers.P1 placing on (0, 1)
New State:
[['1-P2' '2-P1' '1-P2']
 ['1-P1' '1-P3' '1-P34']
 ['1-P34' '1-P34' '----']]
FourPlayers.P2 placing on (0, 0)
New State:
[['1-P2' '1-P2' '----']
 ['2-P2' '2-P2' '2-P2']
 ['1-P34' '1-P34' '----']]
FourPlayers.P3 placing on (0, 2)
New State:
[['1-P2' '1-P2' '1-P3']
 ['2-P2' '2-P2' '2-P2']
 ['1-P34' '1-P34' '----']]
FourPlayers.P4 placing on (2, 0)
New State:
[['----' '2-P34' '1-P3']
 ['1-P34' '3-P34' '2-P2']
 ['1-P34' '2-P34' '----']]
FourPlayers.P2 placing on (0, 0)
New State:

Percentage team wins vs individual wins

In [9]:
sims = [simulate_game() for _ in range(10)]

In [29]:
def get_win_stats(simulations, player):
    num_wins = 0
    for history in simulations:
        last_state = history[-1][1]
        atom_type_board = last_state.atom_type
        atom_type_board = atom_type_board[atom_type_board > 0]

        atom_types = set([FourPlayers(i) for i in atom_type_board.flat])
        if player in atom_types:
            num_wins += 1

    return num_wins


In [40]:
def team_win_together_stats(simulations, team_players):
    # this gives the total number of games that teamed up players has won as a team
    # i.e, the last ball present in the board is a team ball
    p1 = team_players[0]
    p2 = team_players[1]
    p_team = p1 | p2

    return get_win_stats(simulations, p_team)


def team_loose_together_stats(simulations, single_players):
    p1 = single_players[0]
    p2 = single_players[1]
    return get_win_stats(simulations, p1) + get_win_stats(simulations, p2)

def team_individual_only_win(simulations, team_players):
    p1 = team_players[0]
    p2 = team_players[1]
    return get_win_stats(simulations, p1) + get_win_stats(simulations, p2)


In [ ]:
def run_experiments():
    sims_mcts = run


In [11]:
num_simulations = 100
ind_stats = [0, 0, 0, 0]
team_stats = 0

for history in sims:
    # history = simulate_game()
    last_state = history[-1][1]

    utilities = board_utility(last_state.atom_type)
    max_value = max(utilities)
    won_players = [i+1 for i, j in enumerate(utilities) if j == max_value]
    if len(won_players)==1:
        player_id = won_players[0] - 1
        ind_stats[player_id] = ind_stats[player_id] + 1
    else:
        team_stats += 1
        # ind_stats[won_players[0] - 1] = ind_stats[won_players[0] - 1] + 1
        # ind_stats[won_players[1] - 1] = ind_stats[won_players[1] - 1] + 1

In [12]:
ind_stats, team_stats

([0, 1, 1, 0], 8)